<a href="https://colab.research.google.com/github/zzhining/python_ml_dl2/blob/main/3%EC%9E%A5/3_2_4_data_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 기본 파생변수 생성

In [ ]:
import pandas as pd
import numpy as np

# 예시 데이터 생성
sales_data = pd.DataFrame({
    'product': ['노트북', '마우스', '키보드', '모니터', '스피커'],
    'price': [1200000, 25000, 80000, 350000, 120000],
    'quantity': [10, 50, 30, 15, 25],
    'cost': [800000, 15000, 50000, 200000, 80000]
})

print(sales_data)

## 1) 기본 파생변수 생성

In [ ]:
# 총 매출액 계산
sales_data['total_sales'] = sales_data['price'] * sales_data['quantity']

# 총 이익 계산
sales_data['profit'] = (sales_data['price'] - sales_data['cost']) * sales_data['quantity']

# 이익률 계산
sales_data['profit_rate'] = (sales_data['profit'] / sales_data['total_sales']) * 100

print(sales_data[['product', 'total_sales', 'profit', 'profit_rate']])

## 2) assgin 메서드

In [ ]:
# assign 메서드 사용
sales_with_rank = sales_data.assign(
    sales_rank=sales_data['total_sales'].rank(ascending=False)
)
print(sales_with_rank[['product', 'total_sales', 'sales_rank']])

# 2. apply 함수

In [ ]:
# 제품 등급 분류 함수 정의
def classify_product_grade(row):
    if row['profit_rate'] >= 40:
        return 'A급'
    elif row['profit_rate'] >= 35:
        return 'B급'
    else:
        return 'C급'

## 1)  DataFrame에 함수 적용

In [ ]:
# apply를 사용하여 제품 등급 생성
sales_data['grade'] = sales_data.apply(classify_product_grade, axis=1)

## 2) 람다 함수 사용

In [ ]:
# 람다 함수를 사용한 가격 구간 분류
sales_data['price_category'] = sales_data['price'].apply(
    lambda x: '고가' if x >= 300000 else '중가' if x >= 100000 else '저가'
)
print(sales_data[['product', 'profit_rate', 'grade', 'price_category']])

## 3) 컬럼별 통계 계산 (axis=0)

In [ ]:
# 컬럼별 통계 계산 (axis=0)
column_stats = sales_data[['price', 'quantity', 'profit']].apply(np.mean)
print(f"평균값:\n{column_stats}")

# 3. map과 applymap

In [ ]:
print(sales_data)

In [ ]:
# 카테고리 매핑 딕셔너리 생성
category_mapping = {
    '노트북': '컴퓨터',
    '마우스': '주변기기',
    '키보드': '주변기기',
    '모니터': '디스플레이',
    '스피커': '오디오'
}

# map을 사용한 카테고리 변환
sales_data['product_category'] = sales_data['product'].map(category_mapping)

print(sales_data[['product', 'product_category']])

In [ ]:
# 등급별 포인트 매핑
grade_points = {'A급': 100, 'B급': 80, 'C급': 60}
sales_data['grade_points'] = sales_data['grade'].map(grade_points)

print(sales_data[['grade', 'grade_points']])

In [ ]:
# 숫자 데이터에 함수 적용 (소수점 반올림)
numeric_columns = ['profit_rate']
sales_data['profit_rate_rounded'] = sales_data['profit_rate'].map(lambda x: round(x, 1))

print(sales_data[['product', 'profit_rate', 'profit_rate_rounded']])

# 4. 조건부 파생변수 생성 (np.where, np.select)

## 1) np.where

In [ ]:
# np.where를 사용한 이진 분류
sales_data['is_profitable'] = np.where(sales_data['profit'] > 1000000, '고수익', '일반수익')
print(sales_data[['product', 'profit', 'is_profitable']])

## 2) np.select

In [ ]:
# np.select를 사용한 다중 조건 분류
conditions = [
    sales_data['total_sales'] >= 10000000,
    sales_data['total_sales'] >= 5000000,
    sales_data['total_sales'] >= 2000000
]

choices = ['대형거래', '중형거래', '소형거래']

sales_data['transaction_size'] = np.select(conditions, choices, default='미니거래')
print(sales_data[['product', 'total_sales', 'is_profitable']])

## 3) pd.cut

In [ ]:
# pd.cut을 사용한 구간 분할
price_bins = [0, 50000, 200000, float('inf')]
price_labels = ['저가', '중가', '고가']
sales_data['price_segment'] = pd.cut(sales_data['price'], bins=price_bins, labels=price_labels)

print(sales_data[['product', 'price', 'price_segment']])

# 5. 집계 함수를 활용한 파생변수 생성

In [ ]:
# 제품 카테고리별 평균 수익률 계산
sales_data['category_avg_profit_rate'] = sales_data.groupby('product_category')['profit_rate'].transform('mean')

# 제품 카테고리별 총 매출 대비 비중 계산
sales_data['category_total_sales'] = sales_data.groupby('product_category')['total_sales'].transform('sum')
sales_data['sales_share_in_category'] = (sales_data['total_sales'] / sales_data['category_total_sales']) * 100

print(sales_data[['product', 'product_category', 'profit_rate', 'category_avg_profit_rate', 'sales_share_in_category']])

In [ ]:
# 정렬 후 누적 합계 계산 (매출 순위별 누적 매출)
sales_sorted = sales_data.sort_values('total_sales', ascending=False)
sales_sorted['cumulative_sales'] = sales_sorted['total_sales'].cumsum()
sales_sorted['cumulative_sales_ratio'] = (sales_sorted['cumulative_sales'] / sales_sorted['total_sales'].sum()) * 100

print(sales_sorted[['product', 'total_sales', 'cumulative_sales', 'cumulative_sales_ratio']].round(1))